# Earthquake location
Earth Physics  
Marco Scuderi  
**Reminder**: all the scripts are available on [GitHub]('https://github.com/marcoscuderi/Earth-physics')

The aim of this program is to analyze real seismograms from stations close by an earthquake to individuate the Hypocenter of the earthquake.

## Theoretical background 

**Problem set**: We need to find the **epicenter** of an earthquake (i.e. location)

**What do we know**: we can record the waveform that propagates through the Erath at different stations that may or may not be close to the epicenter of the earthquae. From the **seismograms** we can individuate the time arrival of the **P and S waves** ($t_p$ and $t_s$). 

**What I do not know**: I need to calculate the **oring time** ($t_0$) of the earthquake. This is fundamental to calculate the **traveltime** of the waveform:  
$T_p$ = $t_p-t_0$


<div style="display: flex; flex-direction: row; align-items: center; justify-content: space-between;">
    <div style="flex: 1;">

**What tools we can use?**: we need to be clever in using the arrival time of the P and S waves to build the **Wadati diagram** to retrive the origin time. 

$t_s - t_p = t_p(\frac{v_p}{v_s}-1)-t_0(\frac{v_p}{v_s}-1)$  

</div>
    <div style="flex: 1; text-align: right;">
        <img src="Figure_2.png" alt="Your Figure" width="800"/>
    </div>
</div>


#### Now we know the origin time   $t_0$  
This information allows us to calculate the **traveltime**: $T_p = t_p - t_0$  


To find the location I can follow two paths:  
 
- **first solution**  
If I know the velocity of the medium I can calculate the **Distance** as follows: $D = V_p * T_p$ 

We can assume as constant velcoty v=5.5 [km/s] that is characteristic of the carbonates in the upper part of the crust in the appennines (8-10km depth) where seismogenic faults are present.  

- **second solution**  
If we do not know the origin time we can assume a $\frac{v_p}{v_s} = \sqrt{3}$.  In other words we assume a Poissonian medium. 

### Import the necessary packages   
**note** you have to install pyproj  
``` conda install pyproj```

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.stats import linregress 
import pyproj 

%matplotlib qt

### Step one - import the data
In the excercise folder we have the data collected from the three components seismometers (i.e. x,y,z) at each station (GUMA, NRCA, PTQR and TERO).  
**know your data** For each station we have a separate file for each component (E, N, Z) 

In [3]:
stazioni = ['NRCA','TERO','PTQR', 'GUMA']
componenti = ['.E.asc', '.N.asc', '.Z.asc']


In [5]:
import os 
file_N=[]; file_E=[]; file_Z=[]

for i in range(len(stazioni)):
    file_N.append(os.path.join('./data',stazioni[i]+componenti[1]))
    file_E.append(os.path.join('./data',stazioni[i]+componenti[0]))
    file_Z.append(os.path.join('./data',stazioni[i]+componenti[2]))

print(file_N)


['./data/NRCA.N.asc', './data/TERO.N.asc', './data/PTQR.N.asc', './data/GUMA.N.asc']


In [8]:
data = {}
kk = 0
for i in zip(file_N,file_E,file_Z):
    data['%s_N'%stazioni[kk]] = np.loadtxt(i[0])
    data['%s_E'%stazioni[kk]] = np.loadtxt(i[1])
    data['%s_Z'%stazioni[kk]] = np.loadtxt(i[2])
    kk+=1
data

{'NRCA_N': array([[ 9.9999998e-03,  3.4403418e+02],
        [ 2.0000000e-02,  3.7803262e+02],
        [ 2.9999999e-02,  3.8803107e+02],
        ...,
        [ 4.8998999e+02, -6.3628906e+01],
        [ 4.8999999e+02, -8.6630447e+01],
        [ 4.9000999e+02, -1.0663200e+02]]),
 'NRCA_E': array([[ 9.9999998e-03,  3.7465384e+02],
        [ 2.0000000e-02,  3.9765396e+02],
        [ 2.9999999e-02,  4.0465405e+02],
        ...,
        [ 4.8998999e+02, -1.2380201e+02],
        [ 4.8999999e+02, -1.2180190e+02],
        [ 4.9000999e+02, -1.1180178e+02]]),
 'NRCA_Z': array([[9.9999998e-03, 1.9478838e+01],
        [2.0000000e-02, 5.1456013e+01],
        [2.9999999e-02, 5.9433182e+01],
        ...,
        [4.8998999e+02, 5.1956250e+02],
        [4.8999999e+02, 4.7853955e+02],
        [4.9000999e+02, 4.6851672e+02]]),
 'TERO_N': array([[ 9.9999998e-03, -1.1776723e+01],
        [ 2.0000000e-02,  3.8223282e+01],
        [ 2.9999999e-02, -1.6776707e+01],
        ...,
        [ 4.8998999e+02, -7.3873

Now we can plot the waveforms from all the stations

In [16]:

for i in stazioni:
    fig, ax = plt.subplots(nrows=3, figsize=(12,12), sharex=True)
    
    ax[0].title.set_text(i)
    
    ax[0].plot(data['%s_N'%i][:,0],data['%s_N'%i][:,1])

    ax[1].plot(data['%s_E'%i][:,0],data['%s_E'%i][:,1])

    ax[2].plot(data['%s_Z'%i][:,0],data['%s_Z'%i][:,1])
    
    print(i)

NRCA
TERO
PTQR
GUMA


Now we can create variables to store the values of $t_p$ and $t_s$.  
I can also calculate the $t_p-t_s$ difference for every station.
- tp, ts 
- ts-tp

In [18]:
# NRCA
tp_1 = 98.7
ts_1 =105.2
y1 = ts_1-tp_1

#GUMA
tp_2 = 103
ts_2= 112.7
y2 = ts_2-tp_2

#TERO
tp_3 = 96.4
ts_3 = 100.6
y3 = ts_3 - tp_3

# PTQR
tp_4 = 98.4
ts_4 = 103
y4 = ts_4-tp_4


To facilitate data manipulation I can create a pandas dataframe 

In [21]:
junk = { 'stazioni':['NRCA', 'GUMA', 'TERO', 'PTQR'],
        'tp':[tp_1,tp_2,tp_3,tp_4],
        'ts':[ts_1, ts_2,ts_3,ts_4],
        'ts_tp':[y1,y2,y3,y4] }

data = pd.DataFrame(junk)
data


,stazioni,tp,ts,ts_tp
0,NRCA,98.7,105.2,6.5
1,GUMA,103.0,112.7,9.7
2,TERO,96.4,100.6,4.2
3,PTQR,98.4,103.0,4.6


Now we are ready to visualize the data

In [22]:
fig = plt.figure()
plt.scatter(data.tp, data.ts_tp)


To calculate the origin time we need to perform a linear fit to the data and calculate the intercept for y=0

In [26]:
fit = linregress(data.tp, data.ts_tp)
t_0 = - fit[1]/fit[0]

t_0

91.90764529807934

Now we can plot all together

In [28]:
fig = plt.figure()
plt.scatter(data.tp, data.ts_tp,color='r')

x = np.arange(-1,150,1)
y = fit[0]*x+fit[1]

plt.plot(x,y,'r--')

plt.scatter(t_0, 0, color='m', marker='D', s=50)
plt.axhline(y=0, color='k')


Ora stimiamo le distanze dalle varie stazioni e quindi stimiamo la nostra localizzazione partendo dai dati ottenuti dal diagramma di Wadati: 

**Metodo 1: D = Vp * T<sub>p</sub>**. 

In [31]:
v = 5.5 #[km/s]

D_NRCA = v*(data.tp[0]-t_0)

data['distance'] = v*(data.tp-t_0)
data

,stazioni,tp,ts,ts_tp,distance
0,NRCA,98.7,105.2,6.5,37.357951
1,GUMA,103.0,112.7,9.7,61.007951
2,TERO,96.4,100.6,4.2,24.707951
3,PTQR,98.4,103.0,4.6,35.707951


Conoscendo le coordinate delle stazioni in lat e long

In [32]:
lat_NRCA, lon_NRCA = (42.83, 13.11)
lat_TERO, lon_TERO = (42.62, 13.6)
lat_PTQR, lon_PTQR = (42.02, 13.03)
lat_GUMA, lon_GUMA = (43.06, 13.03)

lat_sta = np.array([lat_NRCA, lat_TERO, lat_PTQR, lat_GUMA])
lon_sta = np.array([lon_NRCA, lon_TERO, lon_PTQR, lon_GUMA])



Si può pasare da lat e long a coordinate UTM 

In [33]:
utm33 = pyproj.Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=True)

utmx_sta, utmy_sta = utm33(lon_sta, lat_sta)

print(utmx_sta)

[345523.02718352 385186.65885314 336896.9867287  339582.68147729]


Hypocentro fornito dalla sala sismica INGV per confornto con la nostra localizzazione

In [34]:
lat_hyp = 13.31
lon_hyp = 42.42

utm_x_hypo, utm_y_hypo = utm33(lon_hyp, lat_hyp)

Ora rappresentiamo la distanza della stazioni in pianta e la localizzazione dell'evento fornita dalla sala sismica INGV.

Text(0.5, 0, 'utmX (km)')

<p style='text-align: justify;'>Come si vede la distanza delle stazioni definisce una zona dove potrebbe cadere l'epicentro (area sottesa tra le circonferenze nera, rossa
   e verde), ma allo stesso tempo si osservano delle incertezze nella localizzazione. P.es. l'epicentro calcolato dalla sala sismica INGV, con tecniche più sofisticate, non coincide con l'area di intersezione delle 3 ricronferenze, inoltre la stazione PTQR quasi non interseca alcuna circonferenza. Forse l'approssimazione di v = 5.5 km/s e costante è troppo grande.</p> 

Se non conosciamo il tempo di origine, possiamo localizzare l'evento utilizzando l'equazione sottostante (**Metodo 2**). 

<p style='text-align: justify;'><img src="Fig2.png" alt="Drawing" style="float: right" width="600"><br> Nella relazione di destra, ts - tp (o come lo abbiamo definito noi precedentemente S-P) lo abbiamo già caratterizzato, e per un mezzo Poissoniano abbiamo che Vp/Vs = $\sqrt{3}$. 

Text(0.5, 0, 'utmX (km)')

<p style='text-align: justify;'>In questo caso si nota come la "nostra localizzazione" ben approssima quella della sala sismica. Anche in questo caso vi sono degli errori nella localizzazione che di solito devono essere definiti insieme ai parametri ipocentrali del terremoto. 

# <font color='blue'>Summary </font>

<font color='blue'>
    
Importati sismogrammi delle stazioni e, mediante i tempi di arrivo P ed S, ricostruito il diagramma di Wadati.

Fatta la regressione lineare, fit = linregress(tp, S_P), ed individuato t<sub>0</sub>.</p>

Distanza stazioni ipocentro mediante: Metodo 1) D = Vp * T<sub>p</sub>; Metodo 2) se non si conosce il tempo di origine, assumendo Vp/Vs = $\sqrt{3}$

Prima di calcolare la distanza delle stazioni abbiamo trasformato le coordinate (lon,lat) in km 

    import pyproj
    utm33 = pyproj.Proj(proj='utm',zone=33, ellps='WGS84', preserve_units=False)
